In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sat Sep 11 16:38:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation, Input, InputLayer, Flatten, Conv2D, AveragePooling2D, MaxPool2D, GlobalAveragePooling2D, SeparableConv2D, Conv1D, Conv3D, GlobalAveragePooling3D, AveragePooling3D, DepthwiseConv2D, Conv2DTranspose, Reshape
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.activations import relu
from tensorflow.keras.regularizers import L2
from tensorflow.keras.layers.experimental import preprocessing

from tensorflow_addons.layers import StochasticDepth, GroupNormalization
from tensorflow_addons.optimizers import AdamW, RectifiedAdam

from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score

from keras_unet_collection import models as umodels

In [ ]:
features = np.load('/content/drive/MyDrive/Flood/features.npy').astype(np.float32)
labels = np.load('/content/drive/MyDrive/Flood/labels.npy').astype(np.float32)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, labels.reshape(labels.shape[0], 512*512), train_size=0.8, shuffle=True)
x_train, x_test = tf.constant(x_train), tf.constant(x_test)
y_train, y_test = tf.constant(y_train), tf.constant(y_test)

In [ ]:
class SqueezeAndExcite(tf.keras.layers.Layer):
    def __init__(self, n):
        super().__init__()
        self.n = n

    def build(self, input_shape):

        self.channels = input_shape[-1]

        self.pooling = GlobalAveragePooling2D()
        self.dense_1 = Dense(self.n, activation='relu')
        self.dense_2 =  Dense(self.channels, activation='sigmoid')

    def call(self, x, training=False):

        r = self.pooling(x)
        r = self.dense_1(r, training=training)
        r = self.dense_2(r, training=training)

        r = tf.reshape(r, (-1, 1, 1, self.channels))

        return x * r
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "n": self.n,
            "channels": self.channels,
            "pooling": self.pooling,
            "dense_1": self.dense_1,
            "dense_2": self.dense_2
        })
        return config

class ResidualSEBlock(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation='swish', survival_prob=1, dropout=0, kernel_regularizer=None,  shortcut='identity', reduction=4):
        super().__init__()
        
        self.filters = filters
        self.strides = np.array([strides, strides]) if type(strides) == int else np.array(strides)
        self.activation = activation
        self.survival_prob = survival_prob
        self.dropout = dropout
        self.kernel_regularizer = kernel_regularizer
        self.shortcut = shortcut
        self.shortcut_mapping = None
        self.reduction = reduction

    def build(self, input_shape):

        spatial_dim = np.array([input_shape[1], input_shape[2]])
        channels = input_shape[-1]

        self.batch_norm_1 = BatchNormalization()
        self.activation_1 = Activation(self.activation)
        self.conv_1 = Conv2D(self.filters//4, 1, padding='same', kernel_regularizer=self.kernel_regularizer)
        
        self.batch_norm_2 = BatchNormalization()
        self.activation_2 = Activation(self.activation)
        self.conv_2 =  Conv2D(self.filters//4, 3, strides=self.strides, padding='same', kernel_regularizer=self.kernel_regularizer)

        self.batch_norm_3 = BatchNormalization()
        self.activation_3 = Activation(self.activation)
        self.conv_3 =  Conv2D(self.filters, 1, padding='same', kernel_regularizer=self.kernel_regularizer)

        self.squeeze_and_excite = SqueezeAndExcite(self.filters//self.reduction)
        
        if channels != self.filters or self.strides.prod() != 1: 
          if self.shortcut == 'identity':
            self.projection = AveragePooling2D(pool_size=self.strides, strides=self.strides, padding='same')
            self.shortcut_mapping = lambda x : tf.pad(self.projection(x), [[0, 0], [0, 0], [0, 0], [0, self.filters - channels]])
          if self.shortcut == 'projection':
            self.projection = Conv2D(self.filters, 1, strides=self.strides, padding='same')
            self.shortcut_mapping = lambda x : self.projection(x)
        else:
          self.projection = None
          self.shortcut_mapping = lambda x : x
        
        if self.survival_prob != 1:
          self.stochastic_depth = StochasticDepth(self.survival_prob)
        else:
          self.stochastic_depth = None
        
        if self.dropout != 0:
          self.dropout_layer = Dropout(self.dropout)

    def call(self, x, training=False):
        
        r = self.batch_norm_1(x, training=training)
        r = self.activation_1(r)
        r = self.conv_1(r, training=training)
        
        r = self.batch_norm_2(r, training=training)
        r = self.activation_2(r)
        r = self.conv_2(r, training=training)

        r = self.batch_norm_3(r, training=training)
        r = self.activation_3(r)
        r = self.conv_3(r, training=training)

        r = self.squeeze_and_excite(r)

        x = self.shortcut_mapping(x)

        if self.dropout != 0:
          x = self.dropout_layer(x)

        if self.survival_prob != 1:
          return self.stochastic_depth([x, r], training=training)

        return x + r

    def get_config(self):
        config = super().get_config()
        config.update({
            "filters": self.filters,
            "strides": self.strides,
            "activation": self.activation,
            "survival_prob": self.survival_prob,
            "kernel_regularizer": self.kernel_regularizer,
            "shortcut": self.shortcut,
            "projection": self.projection,
            "shortcut_mapping": self.shortcut_mapping,
            "reduction": self.reduction,
            "batch_norm_1": self.batch_norm_1,
            "activation_1": self.activation_1,
            "conv_1": self.conv_1,
            "batch_norm_2": self.batch_norm_2,
            "activation_2": self.activation_2,
            "conv_2": self.conv_2,
            "batch_norm_3": self.batch_norm_3,
            "activation_3": self.activation_3,
            "conv_3": self.conv_3,
            "squeeze_and_excite": self.squeeze_and_excite,
            "stochastic_depth": self.stochastic_depth,
        })
        return config

In [ ]:
activation = 'swish'
factor = 4

In [ ]:
def encoder(filters, x, n=1, pooling=True):
    for i in range(n):
        x = ResidualSEBlock(filters)(x)
    if pooling:
      p = MaxPool2D(padding='same')(x)
      return x, p
    else:
      return x

In [ ]:
def CLR(x, kernel_size, channels):
  x = Activation(activation)(x)
  x = BatchNormalization()(x)
  x = Conv2D(channels, kernel_size, padding='same')(x)
  return x

def CLRI(x, kernel_size, channels):
  x = Conv2D(channels, kernel_size, padding='same')(x)
  x = BatchNormalization()(x)
  x = Activation(activation)(x)
  return x

In [ ]:
def DSDF(xh, xl, k, w=0.4):
  h_channels, l_channels = xh.shape[-1], xl.shape[-1]
  
  m1h = CLR(xh, 3, k)
  m1l = CLR(xl, 3, k)

  m1hd = Conv2D(k, 3, strides=2, padding='same')(m1h)
  m1lu = Conv2DTranspose(k, 3, strides=2, padding='same')(m1l) 
  m2h = CLR(tf.concat([m1lu, m1h], axis=-1), 3, k)
  m2l = CLR(tf.concat([m1hd, m1l], axis=-1), 3, k)

  m2hd = Conv2D(k, 3, strides=2, padding='same')(m2h)
  m2lu = Conv2DTranspose(k, 3, strides=2, padding='same')(m2l)
  m3h = CLR(tf.concat([m2lu, m1h, m2h], axis=-1), 3, k)
  m3l = CLR(tf.concat([m2hd, m1l, m2l], axis=-1), 3, k)

  m3hd = Conv2D(k, 3, strides=2, padding='same')(m3h)
  m3lu = Conv2DTranspose(k, 3, strides=2, padding='same')(m3l)
  m4h = CLR(tf.concat([m3lu, m1h, m2h, m3h], axis=-1), 3, k)
  m4l = CLR(tf.concat([m3hd, m1l, m2l, m3l], axis=-1), 3, k)

  m4hd = Conv2D(k, 3, strides=2, padding='same')(m4h)
  m4lu = Conv2DTranspose(k, 3, strides=2, padding='same')(m4l)
  m5h = Conv2D(xh.shape[-1], 3, activation=activation, padding='same')(tf.concat([m4lu, m1h, m2h, m3h, m4h], axis=-1))
  m5l = Conv2D(xl.shape[-1], 3, activation=activation, padding='same')(tf.concat([m4hd, m1l, m2l, m3l, m4l], axis=-1))

  return xh + w*m5h, xl + w*m5l

def MSRF(x1, x2, x3, x4, w=0.4):

  r1, r2 = DSDF(x1, x2, int(16/factor))
  r3, r4 = DSDF(x3, x4, int(64/factor))

  r1, r2 = DSDF(r1, r2, int(16/factor))
  r3, r4 = DSDF(r3, r4, int(64/factor))

  r2, r3 = DSDF(r2, r3, int(32/factor))

  r1, r2 = DSDF(r1, r2, int(16/factor))
  r3, r4 = DSDF(r3, r4, int(64/factor))

  r2, r3 = DSDF(r2, r3, int(32/factor))

  r1, r2 = DSDF(r1, r2, int(16/factor))
  r3, r4 = DSDF(r3, r4, int(64/factor))

  return x1 + w*r1, x2 + w*r2, x3 + w*r3, x4 + w*r4

In [ ]:
def RBI(x):
  channels = x.shape[-1]
  r = CLRI(x, 3, channels)
  r = CLRI(r, 3, channels)
  return x + r

def GC(s, x):
  channels = s.shape[-1]
  a = Conv2D(channels, 1, padding='same')(tf.concat([s, x], axis=-1))
  a = Activation('sigmoid')(a)
  s = RBI(s * a)

  return s

def shape_stream(x1, x2, x3, x4, img_gradients):
  spatial_dim = x1.shape[1:3]
  s = CLRI(x1, 1, x1.shape[-1])
  s = RBI(s)
  s = RBI(GC(s, tf.image.resize(x2, spatial_dim)))
  s = RBI(GC(s, tf.image.resize(x3, spatial_dim)))
  s = RBI(GC(s, tf.image.resize(x4, spatial_dim)))
  bce_output = Conv2D(1, 1, activation='sigmoid', padding='same')(s)
  output = CLRI(tf.concat([bce_output, img_gradients[0], img_gradients[1]], axis=-1), 1, 1)
  return bce_output, output

In [ ]:
def RB(x):
  channels = x.shape[-1]
  r = CLR(x, 3, channels)
  r = CLR(r, 3, channels)
  return x + r

def decoder(d, msrf):

  channels = msrf.shape[-1]
  d_up = Conv2DTranspose(channels, 3, strides=2, padding='same')(d)
  x = Conv2D(channels, 3, padding='same')(tf.concat([d_up, msrf], axis=-1))
  spatial = Conv2D(1, 1, activation='sigmoid', padding='same')(x)
  se = SqueezeAndExcite(x.shape[-1]//4)(x)
  att1 = (spatial + 1) * se

  channels = d.shape[-1]
  msrf_down = Conv2D(channels, 3, strides=2, padding='same')(msrf)
  d2 = Conv2D(channels, 1, padding='same')(d)
  x = Activation('relu')(d2 + msrf_down)
  x = Conv2D(1, 1, activation='sigmoid', padding='same')(x)
  x = Conv2DTranspose(1, 3, strides=2, padding='same')(x)
  att2 = msrf * x

  d_up = Conv2DTranspose(msrf.shape[-1], 3, strides=2, padding='same')(d)
  x = Conv2D(channels, 1, padding='same')(tf.concat([att1, att2, d_up], axis=-1))
  x = RB(x)

  return x

In [ ]:
def decoder(d, r):
  channels = r.shape[-1]
  x = Conv2DTranspose(channels, 3, strides=2, padding='same')(d)
  x = Conv2D(channels, 3, padding='same')(tf.concat([x, r], axis=-1))
  x = ResidualSEBlock(x.shape[-1])(x)
  return x

In [ ]:
w = 0.4

x = inp = Input((512, 512, 2))


x = Conv2D(4, 3, strides=2, padding='same')(x)
x = Conv2D(8, 3, strides=2, padding='same')(x)



x1, p1 = encoder(int(64/factor), x)
x2, p2 = encoder(int(128/factor), p1)
x3, p3 = encoder(int(256/factor), p2)
x4 = encoder(int(512/factor), p3, pooling=False)

x1, x2, x3, x4 = MSRF(x1, x2, x3, x4, w=w)

bce_out, ss = shape_stream(x1, x2, x3, x4, img_gradients=tf.image.image_gradients(x))

x3 = decoder(x4, x3)
x2 = decoder(x3, x2)
x1 = decoder(x2, x1)

x = Conv2D(int(64/factor), 3, activation=activation, padding='same')(tf.concat([ss, x1], axis=-1))
x = Conv2D(1, 1, activation='sigmoid', padding='same')(x)



x = Conv2DTranspose(1, 3, strides=2, padding='same')(x)
x = Conv2DTranspose(1, 3, strides=2, activation='sigmoid', padding='same')(x)

bce_out = Conv2DTranspose(1, 3, strides=2, padding='same')(bce_out)
bce_out = Conv2DTranspose(1, 3, strides=2, activation='sigmoid', padding='same')(bce_out)


bce_out = Flatten(name='bce_output')(bce_out)
x = Flatten(name='output')(x)

models = [Model(inputs=inp, outputs=[x, bce_out])]
models[0].summary()

In [ ]:
class AdjustedBinaryCrossentropy(tf.keras.losses.Loss):
  def __init__(self):
    super().__init__()
    self.loss = tf.keras.losses.BinaryCrossentropy()

  def call(self, y_true, y_pred, sample_weight=None):
    mask = tf.cast(y_true != 255, tf.float32)
    y_true = y_true * mask
    y_pred = y_pred * mask
    return self.loss(y_true, y_pred, sample_weight=sample_weight)

class AdjustedBinaryCrossentropy2(tf.keras.losses.Loss):
  def __init__(self):
    super().__init__()
    self.loss = tf.keras.losses.BinaryCrossentropy()

  def call(self, y_true, y_pred, sample_weight=None):

    y_true = tf.reshape(y_true, [-1, 512, 512, 1])
    y_true = tf.cast(tf.reduce_sum(tf.image.sobel_edges(y_true), axis=-1) != 0, tf.float32)
    y_true = tf.reshape(y_true, [-1, 512*512])

    mask = tf.cast(y_true != 255, tf.float32)
    y_true = y_true * mask
    y_pred = y_pred * mask
    return self.loss(y_true, y_pred, sample_weight=sample_weight)
    
class LogCoshDice(tf.keras.losses.Loss):
  def __init__(self):
    super().__init__()
    # self.loss = tf.keras.losses.LogCosh()

  def call(self, y_true, y_pred, sample_weight=None):
    mask = tf.cast(y_true != 255, tf.float32)
    y_true = y_true * mask
    y_pred = y_pred * mask
    dice = 1 - tf.reduce_mean((2 * tf.reduce_sum(y_true * y_pred, axis=-1) + 1)/ (tf.reduce_sum(y_true, axis=-1) + tf.reduce_sum(y_pred, axis=-1) + 1))
    return tf.math.log(tf.cosh(dice) + 1e-7) 

class SoftDice(tf.keras.losses.Loss):
  def __init__(self):
    super().__init__()
    # self.loss = tf.keras.losses.LogCosh()

  def call(self, y_true, y_pred, sample_weight=None):
    mask = tf.cast(y_true != 255, tf.float32)
    y_true = y_true * mask
    y_pred = y_pred * mask
    return tf.reduce_mean(1 - (2 * tf.reduce_sum(y_true * y_pred, axis=-1) + 1) / (tf.reduce_sum(y_true**2, axis=-1) + tf.reduce_sum(y_pred**2, axis=-1) + 1))

class PowerJaccard(tf.keras.losses.Loss):
  def __init__(self, p):
    super().__init__()
    self.p = p

  def call(self, y_true, y_pred, sample_weight=None):
    mask = tf.cast(y_true != 255, tf.float32)
    y_true = y_true * mask
    y_pred = y_pred * mask
    inter = tf.reduce_sum(y_true * y_pred, axis=-1)
    return tf.reduce_mean(1 - (inter + 1e-7)/ (tf.reduce_sum(y_true**self.p, axis=-1) + tf.reduce_sum(y_pred**self.p, axis=-1) - inter + 1e-7))

class AdjustedBinaryCrossentropyAndDiceCoeff(tf.keras.losses.Loss):
  def __init__(self):
    super().__init__()
    self.loss1 = tf.keras.losses.BinaryCrossentropy()
    self.loss2 = LogCoshDice()

  def call(self, y_true, y_pred, sample_weight=None):
    mask = tf.cast(y_true != 255, tf.float32)
    y_true = y_true * mask
    y_pred = y_pred * mask
    return (self.loss1(y_true, y_pred) + self.loss2(y_true, y_pred)) / 2

class AdjustedBinaryCrossentropyAndDiceCoeff2(tf.keras.losses.Loss):
  def __init__(self):
    super().__init__()
    self.loss1 = tf.keras.losses.BinaryCrossentropy()
    self.loss2 = SoftDice()

  def call(self, y_true, y_pred, sample_weight=None):
    mask = tf.cast(y_true != 255, tf.float32)
    y_true = y_true * mask
    y_pred = y_pred * mask
    return (self.loss1(y_true, y_pred) + self.loss2(y_true, y_pred)) / 2

class AdjustedBinaryCrossentropyAndDiceCoeff3(tf.keras.losses.Loss):
  def __init__(self):
    super().__init__()
    self.loss1 = tf.keras.losses.BinaryCrossentropy()

  def call(self, y_true, y_pred, sample_weight=None):
    mask = tf.cast(y_true != 255, tf.float32)
    y_true = y_true * mask
    y_pred = y_pred * mask
    inter = tf.reduce_sum(y_true * y_pred)
    loss2 = inter / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + inter + 1e-7) / (16 * 512 * 512)
    return (self.loss1(y_true, y_pred) + loss2) / 2

def jaccard(y_true, y_pred):
  mask = tf.cast(y_true != 255, tf.float32)
  y_true = tf.cast((y_true * mask) > 0.5, tf.float32)
  y_pred =  tf.cast((y_pred * mask) > 0.5, tf.float32)
  
  n_null = tf.reduce_sum(tf.cast(y_true == 255, tf.float32), axis=1)

  inter = tf.reduce_sum(tf.cast(y_true == y_pred, tf.float32), axis=1)
  return (inter - n_null) / (2*512*512 - inter - n_null)

In [ ]:
batch_size = 32
decay_epochs = 10
epochs = 80

In [ ]:
for i, model in enumerate(models):

  opt = Adam(tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=(x_train.shape[0]//batch_size)*decay_epochs, decay_rate=0.1, staircase=True))
  loss = {
      'output': AdjustedBinaryCrossentropy(),
      # 'bce_output': AdjustedBinaryCrossentropy2()
  }
  metrics = {
      'output': jaccard,
  }

  model.compile(optimizer=opt, loss=loss, metrics=metrics)
  model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs , validation_data=(x_test, y_test))

  # model.save(f'/content/drive/MyDrive/Flood/model_{i+1+num_saved}')

Epoch 1/80
14/14 [==============================] - 8s 282ms/step - loss: 2.0121 - jaccard: 0.3978 - val_loss: 0.7718 - val_jaccard: 0.4675
Epoch 2/80
14/14 [==============================] - 3s 214ms/step - loss: 0.6166 - jaccard: 0.6492 - val_loss: 0.5291 - val_jaccard: 0.7737
Epoch 3/80
14/14 [==============================] - 3s 214ms/step - loss: 0.3415 - jaccard: 0.8468 - val_loss: 0.5712 - val_jaccard: 0.7592
Epoch 4/80
14/14 [==============================] - 3s 217ms/step - loss: 0.2237 - jaccard: 0.8696 - val_loss: 0.4048 - val_jaccard: 0.8299
Epoch 5/80
14/14 [==============================] - 3s 216ms/step - loss: 0.1924 - jaccard: 0.8829 - val_loss: 0.4222 - val_jaccard: 0.8082
Epoch 6/80
14/14 [==============================] - 3s 218ms/step - loss: 0.1884 - jaccard: 0.8850 - val_loss: 1.4781 - val_jaccard: 0.2862
Epoch 7/80
14/14 [==============================] - 3s 219ms/step - loss: 0.2026 - jaccard: 0.8795 - val_loss: 0.2288 - val_jaccard: 0.8888
Epoch 8/80
14/14 [==